In [1]:
import pandas as pd
import numpy as np

In [5]:
from sklearn import linear_model

In [3]:
kc_test = pd.read_csv('c://users/intel/Desktop/Coursera/Regression/week2/kc_house_test_data.csv', dtype= {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int})
kc_train = pd.read_csv('c://users/intel/Desktop/Coursera/Regression/week2/kc_house_train_data.csv', dtype= {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int})

In [4]:
x_train, x_test = kc_train, kc_test

In [13]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
x_train[example_features].head()

,sqft_living,bedrooms,bathrooms
0,1180.0,3.0,1.00
1,2570.0,3.0,2.25
2,770.0,2.0,1.00
3,1960.0,4.0,3.00
4,1680.0,3.0,2.00


In [89]:
model = linear_model.base.LinearRegression(fit_intercept=True)
regr = model.fit(x_train[example_features], x_train['price'])

In [28]:
example_weight_summary = regr.coef_, regr.intercept_
print(example_weight_summary)

(array([   315.40669062, -65081.88711588,   6942.16598637]), 87912.865814964636)


In [36]:
example_predictions = regr.predict(x_train[example_features])
print(example_predictions[0]) # should be 271789.505878

271789.26538


# Compute RSS

In [37]:
def get_residual_sum_of_squares(model, dataset, features, outcome):
    example_predictions = model.predict(dataset[features])
    residuals = example_predictions - dataset[outcome]
    # Then compute the residuals/errors
    RSS = sum(residuals**2)
    # Then square and add them up
    return(RSS)

In [92]:
get_residual_sum_of_squares(regr, x_test, example_features, 'price')

273761940583133.97

# Create new features

In [48]:
from math import log

In [46]:
help(log)

Help on built-in function log in module math:

log(...)
    log(x[, base])
    
    Return the logarithm of x to the given base.
    If the base not specified, returns the natural logarithm (base e) of x.



In [41]:
x_train['bedrooms_squared'] = x_train['bedrooms'].apply(lambda x: x**2)
x_test['bedrooms_squared'] = x_test['bedrooms'].apply(lambda x: x**2)
x_train['bed_bath_rooms'] = x_train['bedrooms']*x_train['bathrooms']
x_test['bed_bath_rooms'] = x_test['bedrooms']*x_test['bathrooms']
x_train['log_sqft_living'] = x_train['sqft_living'].apply(lambda x: log(x))
x_test['log_sqft_living'] = x_test['sqft_living'].apply(lambda x: log(x))
x_train['lat_plus_long'] = x_train['lat'] + x_train['long']
x_test['lat_plus_long'] = x_test['lat'] + x_test['long']

In [50]:
def mean_value(dataset, feature_name):
    avg = dataset[feature_name].mean()
    return(avg)

In [52]:
features = ['bedrooms_squared', 'bed_bath_rooms','log_sqft_living','lat_plus_long']
round(mean_value(x_test, features), 2)

bedrooms_squared    12.45
bed_bath_rooms       7.50
log_sqft_living      7.55
lat_plus_long      -74.65
dtype: float64

# Learning Multiple Models

In [54]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

In [80]:
def models_creation(dataset, model_features):
    model = linear_model.base.LinearRegression(fit_intercept=True)
    model.fit(dataset[model_features], dataset['price'])
    model_weight_summary = model.coef_, model.intercept_
    return(model_weight_summary)

In [81]:
model_name_weight_1 = models_creation(x_train, model_1_features)
model_name_weight_2 = models_creation(x_train, model_2_features)
model_name_weight_3 = models_creation(x_train, model_3_features)
print(model_name_weight_1)
print(model_name_weight_2)
print(model_name_weight_3)

(array([  3.12258646e+02,  -5.95865332e+04,   1.57067421e+04,
         6.58619264e+05,  -3.09374351e+05]), -69075726.792569771)
(array([  3.06610053e+02,  -1.13446368e+05,  -7.14613083e+04,
         6.54844630e+05,  -2.94298969e+05,   2.55796520e+04]), -66867968.871078826)
(array([  5.29422820e+02,   3.45142296e+04,   6.70607813e+04,
         5.34085611e+05,  -4.06750711e+05,  -8.57050439e+03,
        -6.78858667e+03,  -5.61831484e+05,   1.27334900e+05]), -62036084.986098312)


# Comparing multiple models

In [100]:
model1 = linear_model.base.LinearRegression(fit_intercept=True)
model2 = linear_model.base.LinearRegression(fit_intercept=True)
model3 = linear_model.base.LinearRegression(fit_intercept=True)
regr1 = model1.fit(x_train[model_1_features], x_train['price'])
regr2 = model2.fit(x_train[model_2_features], x_train['price'])
regr3 = model3.fit(x_train[model_3_features], x_train['price'])

In [105]:
rss_train = [get_residual_sum_of_squares(regr1, x_train, model_1_features, 'price'),
       get_residual_sum_of_squares(regr2, x_train, model_2_features, 'price'),
       get_residual_sum_of_squares(regr3, x_train, model_3_features, 'price')]
print(rss_train)

[967879963049549.13, 958419635074069.25, 903436455050480.5]


In [106]:
rss_test = [get_residual_sum_of_squares(regr1, x_test, model_1_features, 'price'),
       get_residual_sum_of_squares(regr2, x_test, model_2_features, 'price'),
       get_residual_sum_of_squares(regr3, x_test, model_3_features, 'price')]
print(rss_test)

[225500469795490.31, 223377462976466.31, 259236319207180.0]
